# Sentiment Analysis and Stock Prediction Model

## Retrieve data

In [1]:
import stock_retriever as sr
from datetime import datetime
import os
stocks = ['MSFT', 'AAPL', 'NVDA','GOOGL','GOOG','AMZN','META','TSLA']
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')
for stock in stocks:
    if(not os.path.exists(f'stock_data/{stock}_{start_date}_{end_date}.csv')):
        sr.get_stock_data(stock, start_date=start_date, end_date=end_date).to_csv(f'stock_data/{stock}_{start_date}_{end_date}.csv')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## Deep Learning

In [5]:
import torch
from torch import nn
import math
import pandas as pd

stock = 'MSFT'

stock_data = pd.read_csv(f'stock_data/{stock}_{start_date}_{end_date}.csv')

train_split = int(math.ceil(len(stock_data)*0.8))
test_split = len(stock_data) - train_split

train_data, test_data = stock_data[:train_split].iloc[:,:1], stock_data[train_split:].iloc[:,:1]
print(train_data.shape, test_data.shape)

(853, 1) (213, 1)
